In [131]:
import pandas as pd
import pickle
import os

In [132]:
weather = pd.read_csv(r"..\all_data\data_predictions\weather_trainer.csv")
competitors = pd.read_csv(r"..\all_data\data_predictions\price_of_competitor.csv")
transactions = pd.read_parquet(r"..\all_data\main_data_parq\transaction_df.parquet")


In [133]:
linker = transactions['place'].notnull()
start_len_data = len(transactions)
transactions = transactions[linker]
print("Число удаленных строк с None в place: ", start_len_data - len(transactions))
transactions.info() 

Число удаленных строк с None в place:  432
<class 'pandas.core.frame.DataFrame'>
Int64Index: 874604 entries, 0 to 875035
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   product   874604 non-null  object        
 1   price     874604 non-null  float64       
 2   amount    874604 non-null  float64       
 3   place     874604 non-null  object        
 4   datetime  874604 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 40.0+ MB


In [134]:
linker = ((transactions["price"] > 0) & (transactions["amount"] > 0))
start_len_data = len(transactions)
transactions = transactions[linker]
print("Число удаленных строк с отрицательными значениями price, amount: ", start_len_data - len(transactions))

transactions.describe() 

Число удаленных строк с отрицательными значениями price, amount:  218945


,price,amount
count,655659.000000,655659.000000
mean,11.407969,0.987747
std,5.403327,0.576466
min,2.730000,0.000001
25%,7.490000,0.487804
50%,10.000000,0.982130
75%,14.140000,1.484279
max,31.200000,1.999999


In [135]:
with open(r'..\all_data\few_models\model_cost.pkl', 'rb') as file:
    model_cost = pickle.load(file)
with open(r'..\all_data\few_models\model_amount.pkl', 'rb') as file:
    model_amount = pickle.load(file)

In [136]:
data_costs_model= []

for places in competitors['place'].unique():
    for products in  competitors['product'].unique():
        X = pd.DataFrame({
        "month": pd.date_range(start ='2218-09-28' , periods = 90).strftime('%b'),
        "year" : pd.date_range(start ='2218-09-28' , periods = 90).year})
        X['place'] = places
        X['product'] = products
        data_costs_model.append(X)


data_costs_model = pd.concat(data_costs_model)
data_costs_model   


,month,year,place,product
0,Sep,2218,Анор Лондо,Целебные травы
1,Sep,2218,Анор Лондо,Целебные травы
2,Sep,2218,Анор Лондо,Целебные травы
3,Oct,2218,Анор Лондо,Целебные травы
4,Oct,2218,Анор Лондо,Целебные травы
...,...,...,...,...
85,Dec,2218,Фалькония,Эстус
86,Dec,2218,Фалькония,Эстус
87,Dec,2218,Фалькония,Эстус
88,Dec,2218,Фалькония,Эстус


In [137]:
data_cost_for_pred = data_costs_model.copy()
encoded_place= pd.get_dummies(data_cost_for_pred['place'],prefix = 'place', drop_first=False).astype('int')
encoded_product = pd.get_dummies(data_cost_for_pred['product'],prefix = 'product', drop_first=False).astype('int')
encoded_month = pd.get_dummies(data_cost_for_pred['month'],prefix = 'month', drop_first=False).astype('int')
encoded_year = pd.get_dummies(data_cost_for_pred['year'],prefix = 'year', drop_first=False).astype('int')
data_cost_for_pred = pd.concat([data_cost_for_pred,encoded_product,encoded_place,encoded_month,encoded_year], axis=1)
data_cost_for_pred.drop(columns=['place', 'product','month',"year"], inplace=True)
data_cost_for_pred

,product_Целебные травы,product_Эльфийская пыльца,product_Эстус,place_Анор Лондо,place_Врата Балдура,place_Кеджистан,place_Нокрон,place_Фалькония,month_Dec,month_Nov,month_Oct,month_Sep,year_2218
0,1,0,0,1,0,0,0,0,0,0,0,1,1
1,1,0,0,1,0,0,0,0,0,0,0,1,1
2,1,0,0,1,0,0,0,0,0,0,0,1,1
3,1,0,0,1,0,0,0,0,0,0,1,0,1
4,1,0,0,1,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,0,1,0,0,0,0,1,1,0,0,0,1
86,0,0,1,0,0,0,0,1,1,0,0,0,1
87,0,0,1,0,0,0,0,1,1,0,0,0,1
88,0,0,1,0,0,0,0,1,1,0,0,0,1


In [138]:
for months in pd.date_range(start ='2218-09-28' ,periods = 12, freq = "M").strftime('%b'):
    if 'month_' + months not in data_cost_for_pred:
        name_of_col = 'month_' + months
        
        data_cost_for_pred[name_of_col] = 0

data_cost_for_pred['year_2217'] = 0
data_cost_for_pred['year_2216'] = 0

In [139]:
prediction_costs = model_cost.predict(data_cost_for_pred)

In [140]:
data_costs_model['cost'] = prediction_costs
data_costs_model

,month,year,place,product,cost
0,Sep,2218,Анор Лондо,Целебные травы,3.243147
1,Sep,2218,Анор Лондо,Целебные травы,3.243147
2,Sep,2218,Анор Лондо,Целебные травы,3.243147
3,Oct,2218,Анор Лондо,Целебные травы,4.953873
4,Oct,2218,Анор Лондо,Целебные травы,4.953873
...,...,...,...,...,...
85,Dec,2218,Фалькония,Эстус,3.112093
86,Dec,2218,Фалькония,Эстус,3.112093
87,Dec,2218,Фалькония,Эстус,3.112093
88,Dec,2218,Фалькония,Эстус,3.112093


# Предсказываем значения для amount

In [141]:
data_amount_model= []

for places in competitors['place'].unique():
    for products in  competitors['product'].unique():
        X = pd.DataFrame({
        'date':pd.date_range(start ='2218-09-28' , periods = 90),
        "month": pd.date_range(start ='2218-09-28' , periods = 90).strftime('%b'),
        "year" : pd.date_range(start ='2218-09-28' , periods = 90).year,
        "dates":pd.date_range(start ='2218-09-28' , periods = 90).day })

        X['place'] = places
        X['product'] = products
        data_amount_model.append(X)


data_amount_model = pd.concat(data_amount_model)
data_amount_model   


,date,month,year,dates,place,product
0,2218-09-28,Sep,2218,28,Анор Лондо,Целебные травы
1,2218-09-29,Sep,2218,29,Анор Лондо,Целебные травы
2,2218-09-30,Sep,2218,30,Анор Лондо,Целебные травы
3,2218-10-01,Oct,2218,1,Анор Лондо,Целебные травы
4,2218-10-02,Oct,2218,2,Анор Лондо,Целебные травы
...,...,...,...,...,...,...
85,2218-12-22,Dec,2218,22,Фалькония,Эстус
86,2218-12-23,Dec,2218,23,Фалькония,Эстус
87,2218-12-24,Dec,2218,24,Фалькония,Эстус
88,2218-12-25,Dec,2218,25,Фалькония,Эстус


In [142]:
weather['date'] = pd.to_datetime(weather['date'])
data_amount_model = pd.merge(data_amount_model,weather, on= ['place', 'date'],
         how = 'left')

In [143]:
data_amount_model['cost'] = prediction_costs

In [144]:
data_amount_for_pred = data_amount_model.copy()
encoded_place= pd.get_dummies(data_amount_for_pred['place'],prefix = 'place', drop_first=False).astype('int')
encoded_product = pd.get_dummies(data_amount_for_pred['product'],prefix = 'product', drop_first=False).astype('int')
encoded_month = pd.get_dummies(data_amount_for_pred['month'],prefix = 'month', drop_first=False).astype('int')
encoded_year = pd.get_dummies(data_amount_for_pred['year'],prefix = 'year', drop_first=False).astype('int')
data_amount_for_pred = pd.concat([data_amount_for_pred,encoded_product,encoded_place,encoded_month,encoded_year], axis=1)
data_amount_for_pred.drop(columns=['place', 'product','month',"year",'date','dates'], inplace=True)
data_amount_for_pred

,hot,rain,snow,cost,product_Целебные травы,product_Эльфийская пыльца,product_Эстус,place_Анор Лондо,place_Врата Балдура,place_Кеджистан,place_Нокрон,place_Фалькония,month_Dec,month_Nov,month_Oct,month_Sep,year_2218
0,0,0,0,3.243147,1,0,0,1,0,0,0,0,0,0,0,1,1
1,1,0,0,3.243147,1,0,0,1,0,0,0,0,0,0,0,1,1
2,0,0,0,3.243147,1,0,0,1,0,0,0,0,0,0,0,1,1
3,0,1,0,4.953873,1,0,0,1,0,0,0,0,0,0,1,0,1
4,0,0,0,4.953873,1,0,0,1,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,0,0,0,3.112093,0,0,1,0,0,0,0,1,1,0,0,0,1
1346,0,0,0,3.112093,0,0,1,0,0,0,0,1,1,0,0,0,1
1347,0,0,0,3.112093,0,0,1,0,0,0,0,1,1,0,0,0,1
1348,0,0,1,3.112093,0,0,1,0,0,0,0,1,1,0,0,0,1


In [145]:
for months in pd.date_range(start ='2218-09-28' ,periods = 12, freq = "M" ).strftime('%b'):
    if 'month_' + months not in data_amount_for_pred.columns:
        name_of_col = 'month_' + months
        data_amount_for_pred[name_of_col] = 0

data_amount_for_pred['year_2217'] = 0
data_amount_for_pred['year_2216'] = 0

In [146]:
data_amount_model['comp_prices'] = competitors['concr_price']

In [147]:
data_amount_model['amount'] =  model_amount.predict(data_amount_for_pred)

In [148]:
data_amount_model

,date,month,year,dates,place,product,hot,rain,snow,cost,comp_prices,amount
0,2218-09-28,Sep,2218,28,Анор Лондо,Целебные травы,0,0,0,3.243147,9.386626,39.747064
1,2218-09-29,Sep,2218,29,Анор Лондо,Целебные травы,1,0,0,3.243147,9.394166,50.572522
2,2218-09-30,Sep,2218,30,Анор Лондо,Целебные травы,0,0,0,3.243147,9.401706,39.747064
3,2218-10-01,Oct,2218,1,Анор Лондо,Целебные травы,0,1,0,4.953873,9.360858,52.080418
4,2218-10-02,Oct,2218,2,Анор Лондо,Целебные травы,0,0,0,4.953873,9.368398,38.584262
...,...,...,...,...,...,...,...,...,...,...,...,...
1345,2218-12-22,Dec,2218,22,Фалькония,Эстус,0,0,0,3.112093,15.986168,39.882568
1346,2218-12-23,Dec,2218,23,Фалькония,Эстус,0,0,0,3.112093,15.998137,39.882568
1347,2218-12-24,Dec,2218,24,Фалькония,Эстус,0,0,0,3.112093,16.010106,39.882568
1348,2218-12-25,Dec,2218,25,Фалькония,Эстус,0,0,1,3.112093,16.022076,49.982367


In [149]:
transactions

,product,price,amount,place,datetime
0,Целебные травы,3.90,1.242125,Анор Лондо,2216-01-02 16:28:00
2,Целебные травы,3.90,0.882450,Анор Лондо,2216-01-02 03:03:00
3,Целебные травы,3.90,0.621377,Анор Лондо,2216-01-02 03:12:00
4,Целебные травы,3.90,1.367161,Анор Лондо,2216-01-02 05:33:00
6,Целебные травы,3.90,1.736619,Анор Лондо,2216-01-02 11:54:00
...,...,...,...,...,...
875031,Эстус,9.28,1.079104,Фалькония,2218-09-27 14:50:00
875032,Эстус,9.28,1.580617,Фалькония,2218-09-27 06:16:00
875033,Эстус,9.28,1.738492,Фалькония,2218-09-27 11:14:00
875034,Эстус,9.28,0.430340,Фалькония,2218-09-27 22:24:00


# Итог

Для максимизации прибыли в рамках кейса необходимо поднимать стоимость как можно выше, но при это не нарушая условия. Это обсуловлено тем, что значения amount и cost были спрогнозированы без price, поэтому изменения amount или cost не ожидаются. 

В ходе работы стало ясно, что для прогнозирования цены на товар необходимо знать стоимость товара конкурента(comp_prices)(это значение было спрогнозировано ранее), а также значение цены на товар 3 дня назад, поэтому для реализации использовался цикл while. Каждое новое значение price будет сравниваться со средней стоимостью у конкурента в начале нового трехдневного промежутка умноженного на 1.2, если больше, то будет принимать его значение, если меньше, то к прошлому значению будет прибавляться 1 серебрянный, пока не привысит разницу в 1 золотой с прошлым промежутком или цену по конкурентам умноженную на 1.2. За начальный промежуток была взята последняя известная цена на товар.

In [150]:
dat = []
for places in competitors['place'].unique():
    for products in  competitors['product'].unique():
        daters = {"amount":[], "cost": [], "comp_prices": []}
        dr = data_amount_model[((data_amount_model['place']== places) & (data_amount_model['product'] == products))]
        for i in range(0,len(dr)-1,3):
            daters['cost'].append(dr.iloc[i]['cost'])
            daters['comp_prices'].append(dr.iloc[i]['comp_prices'])
            daters['amount'].append(dr.iloc[i]['amount'] + 
                              dr.iloc[i+1]['amount'] + 
                              dr.iloc[i+2]['amount'])
        x = transactions[((transactions['place']== places) & (transactions['product'] == products))].iloc[-1].price
        print(x)
        prices = []
        price0 = x
        for i in range(30):
            if x > round(1.2 * daters['comp_prices'][i],2):
                x = round(1.2 * daters['comp_prices'][i],2)
                price0 = x
            else: 
                while x < round(1.2 * daters['comp_prices'][i],2) and x - price0 < 1:
                    x = round(x + 0.01,2)
                
            prices.append(round(x,2))
            prices.append(round(x,2))
            prices.append(round(x,2))
            price0 = x
        itog = pd.DataFrame()
        itog['day_number'] = range(1,91)
        itog['product'] = products
        itog['places'] = places
        itog['price'] = prices
        dat.append(itog)

13.629999999999997
18.95
10.680000000000007
16.11000000000001
27.46999999999998
20.32999999999999
19.089999999999993
31.2
24.419999999999995
9.479999999999999
18.109999999999996
9.969999999999999
10.969999999999994
15.279999999999994
9.279999999999998


In [161]:
itog = pd.concat(dat,ignore_index=True)
itog.to_csv(r'..\all_data\itog\schedule_prices.csv', index=False)

# Сверхзадача

In [153]:
data_amount_model['price'] = itog['price']
data_amount_model


,date,month,year,dates,place,product,hot,rain,snow,cost,comp_prices,amount,price
0,2218-09-28,Sep,2218,28,Анор Лондо,Целебные травы,0,0,0,3.243147,9.386626,39.747064,11.26
1,2218-09-29,Sep,2218,29,Анор Лондо,Целебные травы,1,0,0,3.243147,9.394166,50.572522,11.26
2,2218-09-30,Sep,2218,30,Анор Лондо,Целебные травы,0,0,0,3.243147,9.401706,39.747064,11.26
3,2218-10-01,Oct,2218,1,Анор Лондо,Целебные травы,0,1,0,4.953873,9.360858,52.080418,11.23
4,2218-10-02,Oct,2218,2,Анор Лондо,Целебные травы,0,0,0,4.953873,9.368398,38.584262,11.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,2218-12-22,Dec,2218,22,Фалькония,Эстус,0,0,0,3.112093,15.986168,39.882568,19.17
1346,2218-12-23,Dec,2218,23,Фалькония,Эстус,0,0,0,3.112093,15.998137,39.882568,19.17
1347,2218-12-24,Dec,2218,24,Фалькония,Эстус,0,0,0,3.112093,16.010106,39.882568,19.21
1348,2218-12-25,Dec,2218,25,Фалькония,Эстус,0,0,1,3.112093,16.022076,49.982367,19.21


In [159]:
data_amount_model['profit'] = data_amount_model['amount']*(data_amount_model['price'] - data_amount_model['cost'])
data_amount_model['profit'].sum()

811475.6507646097

In [155]:
baseline = pd.read_parquet(r"..\all_data\main_data_parq\df_baseline.parquet")
baseline['cost'] = data_amount_model['cost']
baseline['amount'] = data_amount_model['amount']
baseline['profit'] = baseline['amount']*(baseline['price'] - baseline['cost'])
baseline['profit'].sum()

754609.339754955

In [156]:
print(f'''Удалось увеличить прибыль на {round(data_amount_model['profit'].sum() - baseline['profit'].sum(),2)} золотых''')

Удалось увеличить прибыль на 56866.31 золотых


Таким образом, удалось увеличить прибыль на 56866.31 золотых